In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import omf
import omfvtk
import pandas as pd
import vtki
import PVGeo

In [ ]:
import sys
# This makes the gdc19 package available
sys.path.append('../../')
import gdc19
gdc19.setup_adam()

In [ ]:
# GET THE WELLS DATABASE
WELLS = gdc19.load_well_db()

## Well 58-32 geophysical well log analysis

Lithology:
Basin Fill (0 to 1007 m), Granitoid (1007 to 2296.9 m)

Density of Granite: 2.75 g/cm3

Coordinates: x = 335380.766, y = 4263040.83, z = 1681.61659

Depth: 2296.9 m

In [ ]:
well_log = pd.read_csv(gdc19.get_well_path('58-32_main_geophysical_well_log.csv'))
well_log.head()

In [ ]:
trajectory = pd.read_csv('trajectory_58-32.csv')
trajectory.head()

In [ ]:
# start_x = 335380.766
# start_y = 4263040.83
# count = 0

# theta = []
# phi = []
# r = well_log['Depth (m)'].values
# depth = trajectory['Depth (m)'].values
# dip = trajectory['Dip'].values
# azimuth = trajectory['Azimuth'].values

# #while (count < len(trajectory.index)):
# for i in range (0, len(r)):
#     if (r[i] >= depth[count]) & (r[i] < depth[count+1]):
#         theta.append(dip[count])
#         phi.append(azimuth[count])
#     if (r[i] >= depth[count+1]):
#         count = count + 1
#         theta.append(dip[count])
#         phi.append(azimuth[count])


# r[1:] = r[1:] - r[:-1]

# # convert to radians
# phi = phi * 2 * np.pi / 360.
# theta = (90+theta) * 2 * np.pi / 360.


# x = start_x + (r*np.cos(phi)*np.sin(theta))
# y = start_y + (r*np.sin(phi)*np.sin(theta))
# z = (r*np.cos(theta))

# elevation = np.cumsum(z) + start_z

# #print(len(theta))
# #print(phi)

In [ ]:
x_location = []
y_location = []
elevation = []
DEPTH = well_log['Depth (m)'].values

for i in range (0, len(well_log.index)):
    x = 335380.766
    y = 4263040.83
    z = 1681.61659 - DEPTH[i]
    x_location.append(x)
    y_location.append(y)
    elevation.append(z)

In [ ]:
well_log = well_log.assign(x=pd.Series(x_location).values)
well_log = well_log.assign(y=pd.Series(y_location).values)
well_log = well_log.assign(z=pd.Series(elevation).values)
well_log.head()

In [ ]:
# move XYZ colums to front of table
spac = well_log[['x', 'y', 'z']]
well_poly = PVGeo.pointsToPolyData(spac)
for k, v in well_log.to_dict(orient='list').items():
    well_poly.point_arrays[k] = np.array(v)
well_poly

In [ ]:
well_poly.set_active_scalar('RHOZ')

In [ ]:
well_poly.plot(clim=[0,3.0])

In [ ]:
WELLS['well_5832'] = well_poly
gdc19.save_well_db(WELLS)
WELLS

## Gamma Ray

In [ ]:
gamma = omf.PointSetElement(
    name='gamma ray',
    description='Gamma ray well log of well 58-32. Gamma Ray log data was obtained with the following tools: '\
        'ECGR, GR, GR_EDTC, and HGR. '\
        'For this example, ECGR was the well log chosen. ',
    subtype='point',
    geometry=omf.PointSetGeometry(
        vertices=well_log[['z', 'x', 'y']].values
    ),
    data=[omf.ScalarData(
        name='Gamma Ray Value (API)',
        array=well_log['ECGR'].values,
        location='vertices'
    ),]
)
gamma.validate()

In [ ]:
omfvtk.wrap(gamma).plot(show_edges=False)

## Neutron Porosity

In [ ]:
nphi = omf.PointSetElement(
    name='neutron porosity',
    description='Neutron porosity well log of well 58-32. Neutron Porosity data was obtained with the following tools: '\
        'ECGR, GR, GR_EDTC, and HGR. HTNP_SAN, NPHI, NPOR, HTNP, and TNPH'\
        'For this example, NPHI was the well log chosen. ',
    subtype='point',
    geometry=omf.PointSetGeometry(
        vertices=well_log[['z', 'x', 'y']].values
    ),
    data=[omf.ScalarData(
        name='Porosity (%)',
        array=well_log['NPHI'].values,
        location='vertices'
    ),]
)
nphi.validate()

In [ ]:
omfvtk.wrap(nphi).plot(show_edges=False, rng = (0,1))

## Bulk Density

In [ ]:
density = omf.PointSetElement(
    name='bulk density',
    description='Bulk Density well log of well 58-32. Bulk Density data was obtained with the following tools: '\
        'RHOZ'\
        'For this example, RHOZ was the well log chosen. ',
    subtype='point',
    geometry=omf.PointSetGeometry(
        vertices=well_log[['z', 'x', 'y']].values
    ),
    data=[omf.ScalarData(
        name='Density (g/cc)',
        array=well_log['RHOZ'].values,
        location='vertices'
    ),]
)
density.validate()

In [ ]:
RHOB = omfvtk.wrap(density).plot(show_edges=False, rng = (0, 3))
# RHOB.save('well_5832_RHOB.vtk')